# InsightML

## Library

In [204]:
import csv
import itertools
import json
import numpy as np
import pandas as pd
import pandas_flavor as pf
import re
import ssl
import string
ssl._create_default_https_context = ssl._create_unverified_context

from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import plotly.express as px
import plotly.graph_objects as go

## Magic Format

In [205]:

def about_my_csv(filename):
    # Ouvrir le fichier en mode lecture et détecter le format de délimitation de colonne
    with open(filename, 'r', newline='') as csvfile:
        dialect = csv.Sniffer().sniff(csvfile.read(1024))
    return dialect

def about_my_data(filename):
    # Déterminer le délimiteur
    with open(filename, 'r') as f:
        first_line = f.readline().rstrip('\n')
        # Utilisation de regex pour supprimer le contenu entre les guillemets
        first_line = re.sub(r'(["\'])(?:\\\1|.)*?\1', '', first_line)
        # Si il y a des espaces, c'est forcément entre deux colonnes
        if ' ' in first_line:
            delimiter = None
            space = True
        elif '\t' in first_line:
            delimiter = '\t'
            space = False
        else:
            delimiter = ','
            space = False
    return delimiter, space

def about_my_xls(filename):
    # Lire le fichier Excel en utilisant le module openpyxl et sans en-têtes de colonne
    df_temp = pd.read_excel(filename, engine='openpyxl', header=None)
    # Détecter le format de délimitation de colonne à l'aide de la fonction csv.Sniffer().sniff()
    dialect = csv.Sniffer().sniff(df_temp.to_csv(index=False, header=False))
    return dialect

def about_my_json(filename):
    # Lire le fichier Json pour récuperer la donnée
    with open(filename, 'r') as f:
        data = f.read()
    # Détecter le format du fichier JSON
    try:
        json.loads(data)
        orient = 'records'
    except ValueError:
        orient = 'columns'
    return orient

def about_my_h5(filename):
    # Lire les clés disponibles dans le fichier HDF5
    with pd.HDFStore(filename, mode='r') as store:
        keys = store.keys()

    # Trouver la clé correspondant au plus grand ensemble de données
    max_size = 0
    max_key = None
    for key in keys:
        size = pd.read_hdf(filename, key=key, stop=0).memory_usage(index=True, deep=True).sum()
        if size > max_size:
            max_size = size
            max_key = key
    return max_key

def rename_my_data(df):
    # Renommer les colonnes de AA à ZZ
    new_columns = {}
    for i, col in enumerate(df.columns):
        first_letter = string.ascii_uppercase[i // 26 - 1] if i >= 26 else ''
        second_letter = string.ascii_uppercase[i % 26]
        new_col_name = first_letter + second_letter if first_letter else second_letter
        new_columns[col] = new_col_name

    df = df.rename(columns=new_columns)
    return df


## Importation

In [206]:

path_to_file = 'Datasets/boston.data'

extention = path_to_file.split('.')[-1]

if extention == 'csv':
    # Infos format about my csv.
    my_dialect = about_my_csv(path_to_file)
    # Make dataframe.
    df = pd.read_csv(path_to_file, dialect=my_dialect)

elif extention in ['xls', 'xlsx','xlsm','xlsb']:
    # Infos format about my excel.
    my_dialect = about_my_csv(path_to_file)
    # Make dataframe.
    df = pd.read_excel(path_to_file, engine='openpyxl', delimiter=my_dialect.delimiter)

elif extention == 'json':
    # Infos format about my json.
    my_orient = about_my_csv(path_to_file)
    # Make dataframe.
    df = pd.read_json(path_to_file, orient=my_orient)

elif extention == 'h5':
    # Infos format about my json.
    my_key = about_my_h5(path_to_file)
    # Make dataframe
    df = pd.read_hdf(path_to_file, key=my_key, mode='r')

elif extention == 'data':
    # Infos format about my csv.
    my_dialect, space = about_my_data(path_to_file)
    # Make dataframe
    df = pd.read_csv(path_to_file, dialect=my_dialect, delim_whitespace=space, header=None)
    df = rename_my_data(df)

else: 
    print('format non pris en charge')
    print('Option à venir : SQL, Parquet, Feather, Pickle, HTML, XML')

## Paramètre d'entrée

In [207]:
initial_size = df.shape

target_name = df.columns[-1]

df.dropna(subset=target_name, inplace=True)

df.head()

,A,B,C,D,E,F,G,H,I,J,K
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3
2,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8
4,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8
6,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7
8,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7


## Supp Colonnes

In [208]:
# Calcul du pourcentage de lignes vides pour chaque colonne
pourcentage_lignes_vides = df.isna().sum() / len(df)

# Sélection des colonnes à conserver (celles ayant moins de 30% de lignes vides)
colonnes_a_conserver = pourcentage_lignes_vides[pourcentage_lignes_vides <= 0.3].index

# Création d'un nouveau DataFrame ne contenant que les colonnes à conserver
df = df[colonnes_a_conserver]

## Supp lignes

In [209]:
## Supprime les lignes avec des données sporatiques
# Calculer le pourcentage de valeurs manquantes dans chaque ligne
missing_pct = df.isnull().sum(axis=1) / df.shape[1]
# Calculer le seuil de tolérance en nombre de colonnes sans valeurs
threshold = 0.15 * df.shape[1]
# Supprimer les lignes dont le nombre de colonnes sans valeurs est supérieur au seuil de tolérance
df = df.dropna(thresh=df.shape[1]-threshold)

## DateTime

In [210]:
# Définir la méthode 'detect_date' pour détecter les colonnes de texte contenant des dates
@pf.register_dataframe_method
def detect_date(df):
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            try:
                df[col] = pd.to_datetime(df[col])
            except ValueError:
                pass
    return df


# Détecter et convertir les colonnes de texte en dates
df = df.detect_date()

/var/folders/vh/l2wksfvx397_s94nz191rypm0000gn/T/ipykernel_4001/722678012.py:3: UserWarning:

registration of accessor <class 'pandas_flavor.register.register_dataframe_method.<locals>.inner.<locals>.AccessorMethod'> under name 'detect_date' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.



In [211]:
# Identifier les colonnes de type datetime
date_cols = [col for col in df.columns if df[col].dtype == 'datetime64[ns]']

# Vérifier s'il y a au moins deux colonnes de date
if len(date_cols) < 2:
    print('Il n\'y a pas assez de colonnes de date pour faire des différences de temps.')
else:
    # Normaliser les dates sur UTC
    for date_col in date_cols:
        df[date_col] = df[date_col].dt.tz_convert('Etc/UTC')

    # Créer toutes les combinaisons de colonnes de date
    list_date_cols = list(itertools.combinations(date_cols, 2))

    # Calculer la différence de temps entre chaque paire de colonnes de date
    for paire_date in list_date_cols:
        first_date = paire_date[0]
        second_date = paire_date[1]

        df[f'{first_date}_x_{second_date}'] = (df[second_date] - df[first_date]).dt.total_seconds()


# Extraire : Year, Day, WeekDay, Hour, Minute, Second, Microsecond
for date_col in date_cols:
    # Ajouter les colonnes pour chaque attribut de date
    df[f'{date_col}_year'] = df[date_col].dt.year
    df[f'{date_col}_day_of_year'] = df[date_col].dt.dayofyear
    df[f'{date_col}_day_of_week'] = df[date_col].dt.dayofweek
    df[f'{date_col}_hour'] = df[date_col].dt.hour
    df[f'{date_col}_minute'] = df[date_col].dt.minute
    df[f'{date_col}_second'] = df[date_col].dt.second
    df[f'{date_col}_microsecond'] = df[date_col].dt.microsecond

    df.drop(date_col, axis=1, inplace=True)

Il n'y a pas assez de colonnes de date pour faire des différences de temps.


## Numerical

In [212]:

numeric_features = df.select_dtypes([np.number]).columns

for numerical in numeric_features:
    # Garde uniquement les valeurs qui sont dans l'intervalle [moyenne-2*ecart_type, moyenne+2*ecart_type]
    to_keep = (df[numerical] > df[numerical].mean() - 2*df[numerical].std()) & (df[numerical] < df[numerical].mean() + 2*df[numerical].std())
    df = df.loc[to_keep,:]

    # Transforme en catégoriel quand moins de 8 valeurs différentes.
    if df[numerical].nunique() <= 8:
        df[numerical] = df[numerical].astype(str)

    # Supression valeurs manquantes au dessus de 10 %
    pourcentage_valeur_manquante = 100*df[numerical].isnull().sum()/len(df)
    if pourcentage_valeur_manquante >= 10:
        df.drop(numerical, axis=1, inplace=True)

## Categoriel

In [213]:
categorical_features = df.select_dtypes("object").columns

for categorical in categorical_features:
    # Remplacement des valeurs nulles par la valeur "manquante"
    df[categorical] = df[categorical].fillna('manquante')

    ## Remplace les valeurs sporatique par la valeur "autre"
    counts = df[categorical].value_counts(normalize=True) # Calcul du pourcentage de chaque valeur dans la colonne
    mask = (counts < 0.1) # Sélection des valeurs qui représentent moins de 10% de la colonne
    df[categorical] = df[categorical].replace(counts[mask].index.tolist(), 'autre') # Remplacement des valeurs sélectionnées par la valeur "autre"

    # Suppression des colonnes contenant une seule valeur
    if df[categorical].nunique() == 1:
        df.drop(categorical, axis=1, inplace=True)


#### Par la suite
* Détecter format carte bancaire, indicatif telephone...
* Convertir adresse en format lat.nlong
* Convertir les listes de mots en catégories quand c'est possible.
* ...

## Correction Prè-traitement

In [214]:
# Supprimer les lignes en double
df.drop_duplicates(inplace=True)

## Séparation en variables explicative et cible

In [215]:
Y = df[:][target_name]
X = df.drop(columns=[target_name])

## Suppr colonnes ultra corrélées

In [216]:
corr = X.corr()

high_corr_list = []
cols = corr.columns

for j in cols:
    for i, item in corr[j].iteritems():
        if (i!=j) and abs(item) > 0.9:
            high_corr_list.append((i,j))

no_keep = [high_corr_list[i][0] for i in range(len(high_corr_list)) if i%2 == 0]

X = df.drop(columns=no_keep)

## Séparation en set d'entrainement & de test
Pour le rééquilibrage une meilleur approche est de faire attention :
à la précision, le rappel, la F1-score et l'aire sous la courbe ROC. 

In [217]:
target_dtype = df[target_name].dtype

# Cible Catégoriel
if np.issubdtype(target_dtype, np.object_):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=0, stratify=Y)

    # calculer les pourcentages de chaque valeur unique et les trier par ordre décroissant
    pourcentages = df[target_name].value_counts(normalize=True) * 100
    pourcentages = pourcentages.sort_values(ascending=False)
    # Si l'écart entre la valeur majoritaire et minoritaire est supperieur à 10 %, enclenche le rééchantillonnage
    if pourcentages[0] - pourcentages[-1] < 10 :
        # Rééchantillonnage pour des valeurs Booleennes 
        my_sampling_strategy = 'minority' 
        # Rééchantillonnage pour équilibrer des Classes 
        if len(pourcentages) > 2:
            my_sampling_strategy = 'not majority'
        # Rééchantillonnage 
        oversample = RandomUnderSampler(sampling_strategy=my_sampling_strategy)
        X_train, Y_train = oversample.fit_resample(X_train, Y_train)

# Cible Continue
else: 
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=0)

## Préprocessing Pipeline

In [218]:
# Create pipeline for numeric features
numeric_features = X.select_dtypes([np.number]).columns # Automatically detect positions of numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = X.select_dtypes("object").columns # Automatically detect positions of categorical columns
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train) # Preprocessing influenceur
X_test = preprocessor.transform(X_test) # Preprocessing copieur

## Training Model 

In [219]:
## Define the model to be tuned
# Cible Catégoriel
if np.issubdtype(target_dtype, np.object_):
    model = LogisticRegression()
    modelForest = RandomForestClassifier()

# Cible Continue
else:
    model = LinearRegression()
    modelForest = RandomForestRegressor()

In [220]:
## Regression 
model.fit(X_train, Y_train)

LinearRegression()

In [221]:
## Random Forest
# Define the parameter grid
params = {
    'max_depth': range(4, 11),
    'min_samples_leaf': range(1, 6),
    'min_samples_split': range(2, 9),
    'n_estimators': range(10, 101, 10)
}

# Split the data into a smaller subset for initial testing
small_X_train, small_Y_train = X_train[:100], Y_train[:100]

# Run a randomized search with a smaller number of iterations
n_iter = 10
randomsearch = RandomizedSearchCV(modelForest, param_distributions=params, n_iter=n_iter, cv=2, n_jobs=-1)

# Fit the randomized search on the small dataset
randomsearch.fit(small_X_train, small_Y_train)

# Determine the best hyperparameters from the randomized search
best_params = randomsearch.best_params_

# Refine the parameter grid around the best hyperparameters
params = {
    'max_depth': np.arange(best_params['max_depth'] - 1, best_params['max_depth'] + 2),
    'min_samples_leaf': np.arange(best_params['min_samples_leaf'] - 1, best_params['min_samples_leaf'] + 2),
    'min_samples_split': np.arange(best_params['min_samples_split'] - 1, best_params['min_samples_split'] + 2),
    'n_estimators': np.arange(best_params['n_estimators'] - 20, best_params['n_estimators'] + 20, 10)
}

# Run a grid search with a smaller number of folds
cv = 2
gridsearch = GridSearchCV(modelForest, param_grid=params, cv=cv, n_jobs=-1)

# Fit the grid search on the full dataset
gridsearch.fit(X_train, Y_train)

# Determine the best hyperparameters from the grid search
best_params = gridsearch.best_params_


# Cible Catégoriel
if np.issubdtype(target_dtype, np.object_):
    modelForest = RandomForestClassifier(**best_params)
    
# Cible Continue
else:
    # Train a model on the best hyperparameters
    modelForest = RandomForestRegressor(**best_params)

modelForest.fit(X_train, Y_train)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


120 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 450, in fit
    trees = Parallel(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packa

RandomForestRegressor(max_depth=10, n_estimators=30)

## Predict Model

In [222]:
# Regression 
Y_train_pred = model.predict(X_train) # Predictions on training set
Y_test_pred = model.predict(X_test) # Prédictions on test set

# Random Forest
Y_train_forest_pred = modelForest.predict(X_train) # Predictions on training set
Y_test_forest_pred = modelForest.predict(X_test) # Prédictions on test set 

In [223]:
actual_size = pd.concat([X, Y], axis=1).shape
print(f"Le dataset faisait {initial_size} lignes lors de l'importation, il fait maintenant {actual_size} lignes.")

## Define the model to be tuned
# Cible Catégoriel
if np.issubdtype(target_dtype, np.object_):
    # Train
    ac_regression_train = accuracy_score(Y_train, Y_train_pred)
    ac_rnd_forest_train = accuracy_score(Y_train, Y_train_forest_pred)
    # Test
    ac_regression_test = accuracy_score(Y_test, Y_test_pred)
    ac_rnd_forest_test = accuracy_score(Y_test, Y_test_forest_pred)

    print(f"Sur la regression logistique l'accuracy est de : {ac_regression_train} sur le train et {ac_regression_test} sur le test.")
    print(f"Pendant que la random forest l'accuracy est de : {ac_rnd_forest_train} sur le train et {ac_rnd_forest_test} sur le test.")

# Cible Continue
else:
    # Train
    r2_regression_train = r2_score(Y_train, Y_train_pred)
    r2_rnd_forest_train = r2_score(Y_train, Y_train_forest_pred)
    # Test
    r2_regression_test = r2_score(Y_test, Y_test_pred)
    r2_rnd_forest_test = r2_score(Y_test, Y_test_forest_pred)

    print(f"Sur la regression logistique le r2 est de : {r2_regression_train} sur le train et {r2_regression_test} sur le test.")
    print(f"Pendant que le random forest le r2 est de : {r2_rnd_forest_train} sur le train et {r2_rnd_forest_test} sur le test.")

Le dataset faisait (1012, 11) lignes lors de l'importation, il fait maintenant (320, 10) lignes.
Sur la regression logistique le r2 est de : 1.0 sur le train et 1.0 sur le test.
Pendant que le random forest le r2 est de : 0.9999387260577145 sur le train et 0.9984296139576243 sur le test.


## Affichage

In [224]:
# Cible Catégoriel
if np.issubdtype(target_dtype, np.object_):
    
    conf_matrix_train = confusion_matrix(Y_train, Y_train_pred)
    conf_matrix_test = confusion_matrix(Y_test, Y_test_pred)
    class_names = Y_test.unique().tolist()

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))

    sns.heatmap(conf_matrix_train, annot=True, cmap='Blues', ax=ax1, xticklabels=class_names, yticklabels=class_names)
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('True')
    ax1.set_title('Train Confusion Matrix')

    sns.heatmap(conf_matrix_test, annot=True, cmap='Blues', ax=ax2, xticklabels=class_names, yticklabels=class_names)
    ax2.set_xlabel('Predicted')
    ax2.set_ylabel('True')
    ax2.set_title('Test Confusion Matrix')

    plt.show()

# Cible Continue
else:
    # Visualize the model
    # Visualize predictions on training Set
    fig = px.scatter(x = Y_train, y = Y_train_pred, title = "training set")
    fig.add_trace(go.Scatter(x = Y_train, y = Y_train, name = "real"))
    fig.add_trace(go.Scatter(x = Y_train_pred, y = Y_train_pred, name = "predict"))
    fig.show()

    # Visualize predictions on test Set
    fig = px.scatter(x = Y_test, y = Y_test_pred, title = "test set")
    fig.add_trace(go.Scatter(x = Y_test, y = Y_test, name = "real"))
    fig.add_trace(go.Scatter(x = Y_test_pred, y = Y_test_pred, name = "predict"))
    fig.show()